In [2]:
import json
import os
import time
import requests
from typing import Optional


class VeoVideoGenerator:
  """Complete Veo video generation client using LiteLLM proxy."""
  
  def __init__(self, base_url: str = "https://api.thucchien.ai/gemini/v1beta", 
               api_key: str = "sk-_L5p4n0oPzdMhkMFZh8clA"):
      """
      Initialize the Veo video generator.
      
      Args:
          base_url: Base URL for the LiteLLM proxy with Gemini pass-through
          api_key: API key for LiteLLM proxy authentication
      """
      self.base_url = base_url
      self.api_key = api_key
      self.headers = {
          "x-goog-api-key": api_key,
          "Content-Type": "application/json"
      }
  
  def generate_video(self, prompt: str) -> Optional[str]:
      """
      Initiate video generation with Veo.
      
      Args:
          prompt: Text description of the video to generate
          
      Returns:
          Operation name if successful, None otherwise
      """
      print(f"🎬 Generating video with prompt: '{prompt}'")
      
      url = f"{self.base_url}/models/veo-3.0-generate-preview:predictLongRunning"
      payload = {
          "instances": [{
              "prompt": prompt
          }]
      }
      
      try:
          response = requests.post(url, headers=self.headers, json=payload)
          response.raise_for_status()
          
          data = response.json()
          operation_name = data.get("name")
          
          if operation_name:
              print(f"✅ Video generation started: {operation_name}")
              return operation_name
          else:
              print("❌ No operation name returned")
              print(f"Response: {json.dumps(data, indent=2)}")
              return None
              
      except requests.RequestException as e:
          print(f"❌ Failed to start video generation: {e}")
          if hasattr(e, 'response') and e.response is not None:
              try:
                  error_data = e.response.json()
                  print(f"Error details: {json.dumps(error_data, indent=2)}")
              except:
                  print(f"Error response: {e.response.text}")
          return None
  
  def wait_for_completion(self, operation_name: str, max_wait_time: int = 600) -> Optional[str]:
      """
      Poll operation status until video generation is complete.
      
      Args:
          operation_name: Name of the operation to monitor
          max_wait_time: Maximum time to wait in seconds (default: 10 minutes)
          
      Returns:
          Video URI if successful, None otherwise
      """
      print("⏳ Waiting for video generation to complete...")
      
      operation_url = f"{self.base_url}/{operation_name}"
      start_time = time.time()
      poll_interval = 10  # Start with 10 seconds
      
      while time.time() - start_time < max_wait_time:
          try:
              print(f"🔍 Polling status... ({int(time.time() - start_time)}s elapsed)")
              
              response = requests.get(operation_url, headers=self.headers)
              response.raise_for_status()
              
              data = response.json()
              
              # Check for errors
              if "error" in data:
                  print("❌ Error in video generation:")
                  print(json.dumps(data["error"], indent=2))
                  return None
              
              # Check if operation is complete
              is_done = data.get("done", False)
              
              if is_done:
                  print("🎉 Video generation complete!")
                  
                  try:
                      # Extract video URI from nested response
                      video_uri = data["response"]["generateVideoResponse"]["generatedSamples"][0]["video"]["uri"]
                      print(f"📹 Video URI: {video_uri}")
                      return video_uri
                  except KeyError as e:
                      print(f"❌ Could not extract video URI: {e}")
                      print("Full response:")
                      print(json.dumps(data, indent=2))
                      return None
              
              # Wait before next poll, with exponential backoff
              time.sleep(poll_interval)
              poll_interval = min(poll_interval * 1.2, 30)  # Cap at 30 seconds
              
          except requests.RequestException as e:
              print(f"❌ Error polling operation status: {e}")
              time.sleep(poll_interval)
      
      print(f"⏰ Timeout after {max_wait_time} seconds")
      return None
  
  def download_video(self, video_uri: str, output_filename: str = "generated_video.mp4") -> bool:
      """
      Download the generated video file.
      
      Args:
          video_uri: URI of the video to download (from Google's response)
          output_filename: Local filename to save the video
          
      Returns:
          True if download successful, False otherwise
      """
      print(f"⬇️  Downloading video...")
      print(f"Original URI: {video_uri}")
      
      # Convert Google URI to LiteLLM proxy URI
      # Example: https://generativelanguage.googleapis.com/v1beta/files/abc123 -> /gemini/download/v1beta/files/abc123:download?alt=media
      if video_uri.startswith("https://generativelanguage.googleapis.com/"):
          relative_path = video_uri.replace(
              "https://generativelanguage.googleapis.com/",
              ""
          )
      else:
          relative_path = video_uri

      # base_url: https://api.thucchien.ai/gemini/v1beta
      if self.base_url.endswith("/v1beta"):
          base_path = self.base_url.replace("/v1beta", "/download")
      else:
          base_path = self.base_url

      litellm_download_url = f"{base_path}/{relative_path}"
      print(f"Download URL: {litellm_download_url}")
      
      try:
          # Download with streaming and redirect handling
          response = requests.get(
              litellm_download_url, 
              headers=self.headers, 
              stream=True,
              allow_redirects=True  # Handle redirects automatically
          )
          response.raise_for_status()
          
          # Save video file
          with open(output_filename, 'wb') as f:
              downloaded_size = 0
              for chunk in response.iter_content(chunk_size=8192):
                  if chunk:
                      f.write(chunk)
                      downloaded_size += len(chunk)
                      
                      # Progress indicator for large files
                      if downloaded_size % (1024 * 1024) == 0:  # Every MB
                          print(f"📦 Downloaded {downloaded_size / (1024*1024):.1f} MB...")
          
          # Verify file was created and has content
          if os.path.exists(output_filename):
              file_size = os.path.getsize(output_filename)
              if file_size > 0:
                  print(f"✅ Video downloaded successfully!")
                  print(f"📁 Saved as: {output_filename}")
                  print(f"📏 File size: {file_size / (1024*1024):.2f} MB")
                  return True
              else:
                  print("❌ Downloaded file is empty")
                  os.remove(output_filename)
                  return False
          else:
              print("❌ File was not created")
              return False
              
      except requests.RequestException as e:
          print(f"❌ Download failed: {e}")
          if hasattr(e, 'response') and e.response is not None:
              print(f"Status code: {e.response.status_code}")
              print(f"Response headers: {dict(e.response.headers)}")
          return False
  
  def generate_and_download(self, prompt: str, output_filename: str = None) -> bool:
      """
      Complete workflow: generate video and download it.
      
      Args:
          prompt: Text description for video generation
          output_filename: Output filename (auto-generated if None)
          
      Returns:
          True if successful, False otherwise
      """
      # Auto-generate filename if not provided
      if output_filename is None:
          timestamp = int(time.time())
          safe_prompt = "".join(c for c in prompt[:30] if c.isalnum() or c in (' ', '-', '_')).rstrip()
          output_filename = f"veo_video_{safe_prompt.replace(' ', '_')}_{timestamp}.mp4"
      
      print("=" * 60)
      print("🎬 VEO VIDEO GENERATION WORKFLOW")
      print("=" * 60)
      
      # Step 1: Generate video
      operation_name = self.generate_video(prompt)
      if not operation_name:
          return False
      
      # Step 2: Wait for completion
      video_uri = self.wait_for_completion(operation_name)
      if not video_uri:
          return False
      
      # Step 3: Download video
      success = self.download_video(video_uri, output_filename)
      
      if success:
          print("=" * 60)
          print("🎉 SUCCESS! Video generation complete!")
          print(f"📁 Video saved as: {output_filename}")
          print("=" * 60)
      else:
          print("=" * 60)
          print("❌ FAILED! Video generation or download failed")
          print("=" * 60)
      
      return success


def main():
  """
  Example usage of the VeoVideoGenerator.
  
  Configure these environment variables:
  - LITELLM_BASE_URL: Your LiteLLM proxy URL (default: https://api.thucchien.ai/gemini/v1beta)
  - LITELLM_API_KEY: Your LiteLLM API key (default: sk-_L5p4n0oPzdMhkMFZh8clA)
  """
  
  # Configuration from environment or defaults
  base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/gemini/v1beta")
  api_key = os.getenv("LITELLM_API_KEY", "sk-_L5p4n0oPzdMhkMFZh8clA")
  
  print("🚀 Starting Veo Video Generation Example")
  print(f"📡 Using LiteLLM proxy at: {base_url}")
  
  # Initialize generator
  generator = VeoVideoGenerator(base_url=base_url, api_key=api_key)
  
  # Example prompts - try different ones!
  example_prompts = [
      """An 8-second absolutely seamless audio loop for a podcast background. The transition must be undetectable.

**Mood & Emotion:** Introspective, contemplative, melancholic but with a glimmer of hope. The feeling of a quiet room at dusk, looking at an old black-and-white photograph.

**Instrumentation:** A solo felt piano. The sound should be soft, warm, and intimate.

**Musical Elements:**
- **Tempo:** Very slow, Lento, around 60 bpm.
- **Composition:** A continuous, gentle, arpeggiated chord in C minor that flows without a clear beginning or end. **Crucially, there should be no accented downbeat at the start of the loop.** The musical phrase should feel like it starts mid-thought.
- **Dynamics:** **Extremely consistent soft dynamics (pp).** The attack of each note must be very soft.

**Sound Design & Technical Requirements:**
- **Seamlessness:** Apply a **micro 50ms fade-in at the beginning and a 50ms fade-out at the end** to ensure a perfectly smooth, click-free loop.
- **Reverb:** A touch of warm plate reverb to create space, but the reverb tail must be baked into the loop so it doesn't get cut off.
- **Energy:** The energy level must be perfectly consistent from the final moment of the loop back to the first moment.

**Negative Prompts:** No drums, no bass. No dramatic changes, no loud notes, **no strong attack on the first note.**
""",
  ]
  
  # Use first example or get from user
  prompt = example_prompts[0]
  print(f"🎬 Using prompt: '{prompt}'")
  
  # Generate and download video
  success = generator.generate_and_download(prompt)
  
  if success:
      print("✅ Example completed successfully!")
      print("💡 Try modifying the prompt in the script for different videos!")
  else:
      print("❌ Example failed!")
      print("🔧 Check your API Configuration")

if __name__ == "__main__":
  main()

🚀 Starting Veo Video Generation Example
📡 Using LiteLLM proxy at: https://api.thucchien.ai/gemini/v1beta
🎬 Using prompt: 'An 8-second absolutely seamless audio loop for a podcast background. The transition must be undetectable.

**Mood & Emotion:** Introspective, contemplative, melancholic but with a glimmer of hope. The feeling of a quiet room at dusk, looking at an old black-and-white photograph.

**Instrumentation:** A solo felt piano. The sound should be soft, warm, and intimate.

**Musical Elements:**
- **Tempo:** Very slow, Lento, around 60 bpm.
- **Composition:** A continuous, gentle, arpeggiated chord in C minor that flows without a clear beginning or end. **Crucially, there should be no accented downbeat at the start of the loop.** The musical phrase should feel like it starts mid-thought.
- **Dynamics:** **Extremely consistent soft dynamics (pp).** The attack of each note must be very soft.

**Sound Design & Technical Requirements:**
- **Seamlessness:** Apply a **micro 50ms f

In [3]:
def main():
  """
  Example usage of the VeoVideoGenerator.
  
  Configure these environment variables:
  - LITELLM_BASE_URL: Your LiteLLM proxy URL (default: https://api.thucchien.ai/gemini/v1beta)
  - LITELLM_API_KEY: Your LiteLLM API key (default: sk-_L5p4n0oPzdMhkMFZh8clA)
  """
  
  # Configuration from environment or defaults
  base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/gemini/v1beta")
  api_key = os.getenv("LITELLM_API_KEY", "sk-_L5p4n0oPzdMhkMFZh8clA")
  
  print("🚀 Starting Veo Video Generation Example")
  print(f"📡 Using LiteLLM proxy at: {base_url}")
  
  # Initialize generator
  generator = VeoVideoGenerator(base_url=base_url, api_key=api_key)
  
  # Example prompts - try different ones!
  example_prompts = [
      """A minimalist, 8-second, perfectly seamless video loop for a podcast background.

**Subject & Composition:**
- An abstract, extreme close-up shot looking directly down (top-down view) onto the surface of a spinning CD.
- The frame is filled almost entirely by the disc, making the motion the central focus. We do not see the CD player, only the disc itself against a dark, clean background.

**Lighting & Atmosphere:**
- High-contrast, cinematic lighting (chiaroscuro). A single, cool-toned light source hits the disc from one side, creating a stark interplay between bright, holographic reflections and deep, dark shadows.
- The light catches the tiny grooves of the CD, creating mesmerizing, shifting patterns of rainbow colors.
- The atmosphere is moody, introspective, and profound.

**Action & Mood:**
- The CD spins at a slow, constant, almost mechanical speed. The motion is hypnotic and encourages deep thought.
- The mood is serious, contemplative, and focused, suitable for a deep conversation.

**Technical Requirements:**
- **Camera:** The camera is completely static. No movement whatsoever.
- **Loop:** Must be a mathematically perfect, seamless loop. The rotation and light patterns in the final frame must align perfectly with the first frame.
- **Style:** Hyper-realistic, minimalist, cinematic, abstract. 4K resolution.
""",
  ]
  
  # Use first example or get from user
  prompt = example_prompts[0]
  print(f"🎬 Using prompt: '{prompt}'")
  
  # Generate and download video
  success = generator.generate_and_download(prompt)
  
  if success:
      print("✅ Example completed successfully!")
      print("💡 Try modifying the prompt in the script for different videos!")
  else:
      print("❌ Example failed!")
      print("🔧 Check your API Configuration")

if __name__ == "__main__":
  main()

🚀 Starting Veo Video Generation Example
📡 Using LiteLLM proxy at: https://api.thucchien.ai/gemini/v1beta
🎬 Using prompt: 'A minimalist, 8-second, perfectly seamless video loop for a podcast background.

**Subject & Composition:**
- An abstract, extreme close-up shot looking directly down (top-down view) onto the surface of a spinning CD.
- The frame is filled almost entirely by the disc, making the motion the central focus. We do not see the CD player, only the disc itself against a dark, clean background.

**Lighting & Atmosphere:**
- High-contrast, cinematic lighting (chiaroscuro). A single, cool-toned light source hits the disc from one side, creating a stark interplay between bright, holographic reflections and deep, dark shadows.
- The light catches the tiny grooves of the CD, creating mesmerizing, shifting patterns of rainbow colors.
- The atmosphere is moody, introspective, and profound.

**Action & Mood:**
- The CD spins at a slow, constant, almost mechanical speed. The motion 